This notebook aims to assess the generalization accuracy of a generated suffix, assuming a data-split was used during training.

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

cols_output = ['final_answer_found',
               'final_model_queries', 'final_num_suffixes_checked']

# Load the results + get generated suffixes

In [2]:
# dir_single_query = '/home/chansingh/mntv1/single_query_math_9_17' # single-digit
# dir_single_query = '/home/chansingh/mntv1/single_query_math_double_digit'
# dir_single_query = '/home/chansingh/mntv1/single_query_math_long_suffs' # suffix results
dir_single_query = '/home/chansingh/mntv1/suffix_math_long_suffs_full' # suffix results
r = analyze_utils.load_results_and_cache(dir_single_query, save_file='r_long.pkl', only_keep_scalar=False)
r_single_query = pd.read_pickle(os.path.join(dir_single_query, 'r_long.pkl'))
r_single_query = analyze_utils.postprocess_results(r_single_query)

# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_9_17' # single-digit
# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_double_digit'
# dir_suffix_search = '/home/chansingh/mntv1/suffix_math_long_suffs'
dir_suffix_search = '/home/chansingh/mntv1/suffix_anli_long_suffs_full'
r_suffix_search = analyze_utils.load_results_and_cache(dir_suffix_search, save_file='r_long.pkl', only_keep_scalar=False)
r_suffix_search = pd.read_pickle(os.path.join(dir_suffix_search, 'r_long.pkl'))
r_suffix_search = analyze_utils.postprocess_results(r_suffix_search)

# r = r_single_query
r = pd.concat((r_single_query, r_suffix_search)).sort_values('checkpoint')
# r.to_pickle(f'../results/prompt_gen/results_suffix_math.pkl')

 62%|██████▏   | 26/42 [00:04<00:01,  9.80it/s]

skipping a9fbd677176d7de0bc5e7fa6f6c09007541376f495ed9bdf8d6a67dd34a1e7b9arwz


 76%|███████▌  | 32/42 [00:05<00:01,  7.57it/s]

skipping b88714fc661f1184d2c9edfc455f2e7e9ab4da71e50819725cc9082872293584kgkl


100%|██████████| 37/37 [00:04<00:00,  8.26it/s]


# Look at top suffix candidates

In [46]:
r[r.max_num_tokens == 6]

,task_name,n_shots,max_dset_size,max_digit,template_num_init_string,template_num_task_phrasing,checkpoint,max_num_tokens,beam_size,beam_size_extra,...,Recall @ 30 suffixes,Recall @ 40 suffixes,Recall @ 50 suffixes,Recall @ 75 suffixes,Recall @ 100 suffixes,Recall @ 150 suffixes,Recall @ 200 suffixes,reciprocal_rank,reciprocal_rank_multi,train_split_frac
2,task1146_country_capital,5,1000,10,0,0,EleutherAI/gpt-j-6B,6,5,50,...,False,False,False,False,False,False,True,0.005747,0.015152,1
17,task1146_country_capital,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,False,0.001808,0.001845,1
30,task1147_country_currency,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,True,0.005102,0.015385,1
5,task1509_evalution_antonyms,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,False,0.001031,0.002410,1
1,multiply_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,False,0.002326,0.012346,1
8,max_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,True,0.006452,0.011905,1
12,subtract_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,True,True,0.007143,0.014286,1
14,divide_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,False,False,False,False,False,0.004329,0.004854,1
19,first_two,1,1000,10,0,0,gpt2-xl,6,5,50,...,False,False,True,True,True,True,True,0.020408,0.020000,1
26,double_one,1,1000,10,0,0,gpt2-xl,6,5,50,...,True,True,True,True,True,True,True,0.090909,0.083333,1


In [ ]:
r[['suffix_str_added', 'final_answer_added']].values

In [25]:
args = r.iloc[1]



In [ ]:
r0 = r[(r.max_num_tokens == 6) * (r.n_shots == 1)]
for i in range(r0.shape[0]):
    row = r0.iloc[i]
    print('task', row.task_name)
    candidates, probs = analyze_utils.get_top_candidates_and_probs_suff(row)
    for j in range(5):
        print('\t', candidates[j], probs[j])

# Actually test acc

In [34]:
np.random.seed(args.seed)
(dset, dset_test), check_answer_func, descr = data.get_data(
            args, args.task_name, n_shots=args.n_shots, train_split_frac=args.train_split_frac)

In [35]:
args.task_name

'subtract_two'

In [36]:
dset[0]

{'text': 'Given the input numbers 8 and 0, the answer is 8.\n\n',
 'input': 'Given the input numbers 8 and 0, the answer is',
 'output': ' 8.\n\n',
 '__index_level_0__': 80}

In [28]:
checkpoint = 'EleutherAI/gpt-j-6B' # 'gpt2-medium'

In [29]:
model = prompt_classification.create_model(checkpoint)

In [37]:
# todo: work with proper padding
# todo: work with multitoken outputs
loss, acc = prompt_classification.test_model_on_task_with_prefix(
    dset=dset, model=model, prefix=f'Subtract these numbers: '
)

[('Subtract these numbers: Given the input numbers 8 and 0, the answer is', ' 8.\n\n'), ('Subtract these numbers: Given the input numbers 8 and 4, the answer is', ' 4.\n\n')]
Percent correct: 40.00
